# This code finds the average lick rate per second and plots it against time 

I want to plot the average lick rate against time to see if the duration before they start anticipatorily licking is fixed between trials or if it is a matter if distance passed

In [1]:
import pandas as pd
df = pd.read_csv('wfC318/Day_0/Raw_files_csv/combined_behavior_and_s.csv', header=None)

#adding labels to the data
labs = ['frame', 'time', 'odor_valve', 'total_licks', 'total_rewards', 'trial', 
        'valve_open', 'virt_dist', 'total_distance', 'lap', 'trial_type', 'speed']

#name the rest of the columns with neuron index
for i in range(0,255):
    labs.append(str(i))
    
df.columns = labs
print(df.head())


   frame  time  odor_valve  total_licks  total_rewards  trial  valve_open  \
0      1    34           0            0              0      0           0   
1      2    84           0            0              0      0           0   
2      3   133           0            0              0      0           0   
3      4   183           0            0              0      0           0   
4      5   233           0            0              0      0           0   

   virt_dist  total_distance  lap ...   245     246  247  248  249  250  251  \
0         13               0    0 ...   0.0  0.0000  0.0  0.0  0.0  0.0  0.0   
1         26               0    0 ...   0.0  2.7654  0.0  0.0  0.0  0.0  0.0   
2         36               0    0 ...   0.0  0.0000  0.0  0.0  0.0  0.0  0.0   
3         50               0    0 ...   0.0  0.0000  0.0  0.0  0.0  0.0  0.0   
4         65               0    0 ...   0.0  0.0000  0.0  0.0  0.0  0.0  0.0   

   252  253  254  
0  0.0  0.0  0.0  
1  0.0  0.0  0.0  

In [2]:
#remove all trials after 30
df = df[df["trial"] < 31]

#restart time so it increments within a given trial only
#do not run this cell more than once or it will fuck everything up
trial_prev, time_prev, end_time = 0,0,0
for i, j in df.iterrows(): 
    trial = int(j['trial'])
    time = int(j['time'])
    if trial != trial_prev:
        end_time = time_prev
        #print("\n")
    df.at[i,'time'] = time - end_time
    trial_prev = trial
    time_prev = time

In [11]:
import pandas as pd
import numpy as np
import time as t

# Things to set:
#filename = 'wfC318/Day_4/raw_files_csv/combined_behavior_and_s_COPY.csv'
#filename = 'wfC318/Day_4/raw_files_csv/combined_behavior_and_s.csv'
filename = 'wfC318/Day_0/Raw_files_csv/combined_behavior_and_s.csv'

# How many trials we are looking at (ex: 31 for day 0, 34 for day 4)
trial_count = 31
#trial_count = max(lap)  # all of them


data = pd.read_csv(filename, header=None)


# Will change neuron firing data to (0,1) binary indicators
data_for_lick = pd.read_csv(filename, header=None)

def changeTime(data_name):
    trial_prev, time_prev, end_time = 0,0,0
    for i, j in data_name.iterrows():
        trial = int(j[9])
        time = int(j[1])
        if trial != trial_prev:
            end_time = time_prev
        data_name.at[i,1] = time - end_time
        trial_prev = trial
        time_prev = time
    print("time changed")
    
changeTime(data)
changeTime(data_for_lick)



length = len(data)
data = data[data[7] <= 4000]  # removing all rows where mouse is licking reward past 4000cm
data_for_lick = data_for_lick[data_for_lick[7] <= 4000]  # removing all rows where mouse is licking reward past 4000cm


time = data[1]
odor = data[10]
distance = data[7]
lap = data[9]
lick = data[3]

lap_dictionary = {}
for i in range(0, trial_count):  # for every lap
    lap_dictionary[i] = data.loc[data[9] == i]  # associate all lap data with that lap


#find duration of the trial 
lap_dictionary = {}
for i in range(0, trial_count):  # for every lap
    lap_dictionary[i] = data.loc[data[9] == i]  # associate all lap data with that lap

def lengthofTrial():
    duration = lap_dictionary[0]
    first = lap_dictionary[0].iloc[0]
    last = lap_dictionary[0].iloc[-1]
    duration = last[1] - first[1]
print("duration of trial", duration)


    
#bin the trial 
#find the average lick per second over the trial 
#Plot line graph of average licks per second against time bin 



#(pick a fast and slow trial and compare results)






#bin in 200ms bins 
#num_of_bins = int(time // 200)
#print("number of bins:", num_of_bins)


#find amount of licks per bin  and store as variable 
#def lick_time (total licks, time):

    # plot the data
    #data.iloc[0:581]. plt.plot(time, lick, alpha=0.4, label="no odor")
    #plt.legend()
 
    # show the plot
    #plt.show()
 
    #return "Done!"



#plot as y axis 
    #find delta lick in 200ms 
    #divide by 
#licks per milisecond 
#find delta licks and divide by number of seconds that have passed 
#time bins of 200ms 


time changed
time changed


NameError: name 'duration' is not defined